In [1]:
from openai import OpenAI
import json
import minsearch

In [22]:
with open ('/Users/dmitrywer/Desktop/my_projects/LLM_Zoomcamp/documents.json','r') as file:
    docs_raw = json.load(file)

In [23]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
index.fit(documents)

In [24]:
index.search('Setting up dbt locally with Docker and Postgres?', num_results=2)

[{'text': 'This is not a FAQ but more of an advice if you want to set up dbt locally, I did it in the following way:\nI had the postgres instance from week 2 (year 2024) up (the docker-compose)\nmkdir dbt\nvi dbt/profiles.yml\nAnd here I attached this content (only the required fields) and replaced them with the proper values (for instance mine where in the .env file of the folder of week 2 docker stuff)\ncd dbt && git clone https://github.com/dbt-labs/dbt-starter-project\nmkdir project && cd project && mv dbt-starter-project/* .\nMake sure that you align the profile name in profiles.yml with the dbt_project.yml file\nAdd this line anywhere on the dbt_project.yml file:\nconfig-version: 2\ndocker run --network=mage-zoomcamp_default --mount type=bind,source=/<your-path>/dbt/project,target=/usr/app --mount type=bind,source=/<your-path>/profiles.yml,target=/root/.dbt/profiles.yml ghcr.io/dbt-labs/dbt-postgres ls\nIf you have trouble run\ndocker run --network=mage-zoomcamp_default --mount t

In [8]:
def search(question):
    boost = {'question' : 3.0, 'section' : 0.5}
    results = index.search(query = question, num_results = 3, boost_dict = boost, filter_dict = {'course' : 'data-engineering-zoomcamp'})
    return results

In [9]:
question = 'Setting up dbt locally with Docker and Postgres'

In [10]:
search(question)

[{'text': 'This is not a FAQ but more of an advice if you want to set up dbt locally, I did it in the following way:\nI had the postgres instance from week 2 (year 2024) up (the docker-compose)\nmkdir dbt\nvi dbt/profiles.yml\nAnd here I attached this content (only the required fields) and replaced them with the proper values (for instance mine where in the .env file of the folder of week 2 docker stuff)\ncd dbt && git clone https://github.com/dbt-labs/dbt-starter-project\nmkdir project && cd project && mv dbt-starter-project/* .\nMake sure that you align the profile name in profiles.yml with the dbt_project.yml file\nAdd this line anywhere on the dbt_project.yml file:\nconfig-version: 2\ndocker run --network=mage-zoomcamp_default --mount type=bind,source=/<your-path>/dbt/project,target=/usr/app --mount type=bind,source=/<your-path>/profiles.yml,target=/root/.dbt/profiles.yml ghcr.io/dbt-labs/dbt-postgres ls\nIf you have trouble run\ndocker run --network=mage-zoomcamp_default --mount t

In [51]:
def build_prompt(question, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION. If there is no answer, return NONE.
    
QUESTION: {question}
    
CONTEXT: 
{context}
""".strip()
    context = ''
    for doc in search_results:
        context = context + f'section: {doc['section']}\nquestion: {doc['question']}\ncontext: {doc['text']}\n\n'
    return prompt_template.format(question = question, context = context)
    

In [54]:
question = 'Setting up dbt locally with Docker and Postgres?'
search_results = search(question)
prompt = build_prompt(question, search_results)

In [55]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION. If there is no answer, return NONE.
    
QUESTION: Setting up dbt locally with Docker and Postgres?
    
CONTEXT: 
section: Project
question: Setting up dbt locally with Docker and Postgres
context: This is not a FAQ but more of an advice if you want to set up dbt locally, I did it in the following way:
I had the postgres instance from week 2 (year 2024) up (the docker-compose)
mkdir dbt
vi dbt/profiles.yml
And here I attached this content (only the required fields) and replaced them with the proper values (for instance mine where in the .env file of the folder of week 2 docker stuff)
cd dbt && git clone https://github.com/dbt-labs/dbt-starter-project
mkdir project && cd project && mv dbt-starter-project/* .
Make sure that you align the profile name in profiles.yml with the dbt_project.yml file
Add this line anywhere on the

In [56]:
def llm(prompt):
    client = OpenAI()
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = [{'role' : 'user', 'content' : prompt}])
    return response.choices[0].message.content

In [57]:
llm(prompt)

'To set up dbt locally with Docker and Postgres, you can follow the advice provided:\n\n1. **Postgres Instance Setup:**\n   - Ensure you have the Postgres instance from week 2 up and running using the docker-compose.\n\n2. **Directory and File Preparation:**\n   - Create a directory for dbt:\n     ```bash\n     mkdir dbt\n     ```\n\n   - Create a `profiles.yml` file in the dbt directory and populate it with the necessary fields, replacing the placeholders with appropriate values (you might find these values in the `.env` file of the week 2 docker folder).\n\n     Example:\n     ```bash\n     vi dbt/profiles.yml\n     ```\n\n3. **Clone dbt Starter Project:**\n   - Navigate to the dbt directory and clone the starter project:\n     ```bash\n     cd dbt && git clone https://github.com/dbt-labs/dbt-starter-project\n     ```\n\n   - Create a project directory and move the cloned files:\n     ```bash\n     mkdir project && cd project && mv dbt-starter-project/* .\n     ```\n\n4. **Configurat